In [ ]:
from nbdev import *
# default_exp deployment

In [ ]:
# export
import flask
from flask import Flask
import requests
import json
from flask import request, jsonify
from tempfile import mkdtemp
import os.path as path
from securereqnet.preprocessing import vectorize_sentences
import numpy as np
from waitress import serve

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Jason\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Backend Service
These methods ping the TFX serving endpoint and get predictions from the model.  The input of the served model would be vectorized sentences in tensor form and the output is the output of the final FC layer

In [ ]:
#export

def __get_predictions(sentences, endpoint_uri):
    payload = {
            "instances": vectorize_sentences(sentences).tolist()
        }
        
    print(payload)
    r = requests.post(endpoint_uri, json = payload)
    model_preds = json.loads(r.content.decode('utf-8'))

    
    preds = []

    # decode predictions
    for pred in model_preds['predictions']:
        preds.append(__decode(pred))

    output = {
        "predictions": preds
    }

    return output




In [ ]:
#export

def __decode(input):
    return float(input[0])>float(input[1])

## Flask Backend
Defines a factory for our app.  Serving is packaged nicely inside a serve method, and is ready to deploy with waitress.

In [ ]:
#export

def create_app(test_config=None):
    app = Flask(__name__, instance_relative_config=True)
    app.config.from_mapping(
        SECRET_KEY='dev',
        TFX_ENDPOINT='http://localhost:8503/v1/models/alpha:predict'
    )

    if test_config:
        app.config.from_mapping(test_config)
    else:
        app.config.from_pyfile('config.py', silent=True)

    # default route, we probably will get rid of this
    @app.route('/', methods=['GET'])
    def home():
        return '<h1>SecureReqNet</h1><p>Flask backend</p>'

    # alpha model
    @app.route('/models/alpha', methods=['POST'])
    def alpha():
        content = request.get_json()
        print(content['instances'])
        sentences = content['instances']
        return __get_predictions(sentences, app.config['TFX_ENDPOINT'])

    return app




In [ ]:
#export

def serve(host, port):
    serve(create_app(), host = host, port = port)

# Testing ✅
## Flask Backend

In [ ]:
#all_deployment

In [ ]:
import securereqnet
dir(securereqnet)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__',
 'preprocessing',
 'utils']

In [ ]:
import unittest.mock
from unittest import mock
from unittest.mock import patch
from securereqnet.deployment import create_app

In [ ]:
# Set up test client
app = create_app({'TESTING': True, "TFX_ENDPOINT": "MockMockMock"})
test_client = app.test_client()

In [ ]:
# This tests to see if our app successfully deploys with a homepage
# Succeeds if the response code for default path is 200
def test_app_homepage():
    response = test_client.get('/')
    assert response.status_code==200

test_app_homepage()

In [ ]:
# This test checks if we are able to successfully post a sentence to the server and get a response back
# Mock the backend request to TFX serving
# Succeeds if data returns successfully in correct form
@patch('securereqnet.deployment.__get_predictions')
def test_get_prediction_single(mock_predictions):
    test_payload = {"instances": ["test test test test"]}
    expected_pred = {"predictions": [True]}
    mock_predictions.return_value = expected_pred
    response = test_client.post('/models/alpha', json=test_payload)
    r_data = json.loads(response.data.decode('utf-8'))
    assert r_data == expected_pred

test_get_prediction_single()

['test test test test']


In [ ]:
# This test checks if we are able to successfully post multiple sentences to the server and get a response back
# Mock the backend request to TFX serving
# Succeeds if data returns successfully in correct form
@patch('securereqnet.deployment.__get_predictions')
def test_get_prediction_multi(mock_predictions):
    test_payload = {"instances": ["test test test test", "more testing", "super duper testing"]}
    expected_pred = {"predictions": [True, False, True]}
    mock_predictions.return_value = expected_pred
    response = test_client.post('/models/alpha', json=test_payload)
    r_data = json.loads(response.data.decode('utf-8'))
    assert r_data == expected_pred

test_get_prediction_multi()

['test test test test', 'more testing', 'super duper testing']


## Backend Service